In [ ]:
import math
from itertools import product

In [ ]:
def deg2num(lat_deg, lon_deg, zoom):
    '''
        Convert latitude, longitude to (x,y) tile coordinate at given zoom.
    '''
    lat_rad = math.radians(lat_deg)
    n = 2.0**zoom
    xtile = int((lon_deg + 180.0) / 360.0 * n)
    ytile = int((1.0 - math.asinh(math.tan(lat_rad)) / math.pi) / 2.0 * n)
    return (xtile, ytile)

In [ ]:
def num2deg(xtile, ytile, zoom):
    '''
        Convert (x,y) to latitude, longitude tile coordinate at given zoom.
    '''
    n = 2.0**zoom
    lon_deg = xtile / n * 360.0 - 180.0
    lat_rad = math.atan(math.sinh(math.pi * (1 - 2 * ytile / n)))
    lat_deg = math.degrees(lat_rad)
    return (lat_deg, lon_deg)

In [ ]:
def tiles_url(zoom, lat1, lon1, lat2, lon2):
    ''' 
        Convert geographic bounds into a list of tile_url coordinates at given zoom.
    '''
    # convert to geographic bounding box
    minlat, minlon = min(lat1, lat2), min(lon1, lon2)
    maxlat, maxlon = max(lat1, lat2), max(lon1, lon2)

    # convert to tile-space bounding box
    xmin, ymin = deg2num(maxlat, minlon, zoom)
    xmax, ymax = deg2num(minlat, maxlon, zoom)

    # generate a list of tiles_url
    xs, ys = range(xmin, xmax + 1), range(ymin, ymax + 1)
    tiles_url = [(f"s3://elevation-tiles-prod//terrarium/{zoom}/{x}/{y}.png")
                 for (y, x) in product(ys, xs)]

    return tiles_url

In [ ]:
data = tiles_url(7, 30, -85, 24, -79)

In [6]:
print(data)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['s3://elevation-tiles-prod//terrarium/7/33/52.png', 's3://elevation-tiles-prod//terrarium/7/34/52.png', 's3://elevation-tiles-prod//terrarium/7/35/52.png', 's3://elevation-tiles-prod//terrarium/7/33/53.png', 's3://elevation-tiles-prod//terrarium/7/34/53.png', 's3://elevation-tiles-prod//terrarium/7/35/53.png', 's3://elevation-tiles-prod//terrarium/7/33/54.png', 's3://elevation-tiles-prod//terrarium/7/34/54.png', 's3://elevation-tiles-prod//terrarium/7/35/54.png', 's3://elevation-tiles-prod//terrarium/7/33/55.png', 's3://elevation-tiles-prod//terrarium/7/34/55.png', 's3://elevation-tiles-prod//terrarium/7/35/55.png']

In [7]:
df = spark.read.format("image").load(data)
df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|               image|
+--------------------+
|[s3://elevation-t...|
|[s3://elevation-t...|
|[s3://elevation-t...|
|[s3://elevation-t...|
|[s3://elevation-t...|
|[s3://elevation-t...|
|[s3://elevation-t...|
|[s3://elevation-t...|
|[s3://elevation-t...|
|[s3://elevation-t...|
|[s3://elevation-t...|
|[s3://elevation-t...|
+--------------------+

In [8]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- image: struct (nullable = true)
 |    |-- origin: string (nullable = true)
 |    |-- height: integer (nullable = true)
 |    |-- width: integer (nullable = true)
 |    |-- nChannels: integer (nullable = true)
 |    |-- mode: integer (nullable = true)
 |    |-- data: binary (nullable = true)

In [9]:
df.select("image.origin", "image.height", "image.width", "image.mode",
          "image.nChannels").show(5, truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------------------------------+------+-----+----+---------+
|origin                                         |height|width|mode|nChannels|
+-----------------------------------------------+------+-----+----+---------+
|s3://elevation-tiles-prod/terrarium/7/33/55.png|256   |256  |16  |3        |
|s3://elevation-tiles-prod/terrarium/7/34/55.png|256   |256  |16  |3        |
|s3://elevation-tiles-prod/terrarium/7/35/55.png|256   |256  |16  |3        |
|s3://elevation-tiles-prod/terrarium/7/33/52.png|256   |256  |16  |3        |
|s3://elevation-tiles-prod/terrarium/7/34/52.png|256   |256  |16  |3        |
+-----------------------------------------------+------+-----+----+---------+
only showing top 5 rows

In [10]:
df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

12

In [11]:
# Pobranie dodatkowych paczek python-owych
sc.uninstall_package('pip')
# Upgrade pip
sc.install_pypi_package("pip==22.2.2")
sc.install_pypi_package("opencv-python") #Install opencv-python 
sc.list_packages()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Uninstalling pip-9.0.1:
  Successfully uninstalled pip-9.0.1

  Using cached pip-22.2.2-py3-none-any.whl (2.0 MB)

  Using cached opencv_python-4.6.0.66-cp36-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (60.9 MB)

Package                    Version
-------------------------- ----------
aws-cfn-bootstrap          2.0
beautifulsoup4             4.9.3
boto                       2.49.0
click                      8.1.1
docutils                   0.14
jmespath                   1.0.0
joblib                     1.1.0
lockfile                   0.11.0
lxml                       4.8.0
mysqlclient                1.4.2
nltk                       3.7
nose                       1.3.4
numpy                      1.20.0
opencv-python              4.6.0.66
pip                        22.2.2
py-dateutil                2.2
pystache                   0.5.4
python-daemon              2.2.3
python37-sagemaker-pyspark 1.4.1
pytz                       2022.1
PyYAML                     5.4.1
regex       

In [ ]:
import cv2